<a href="https://colab.research.google.com/github/yzhao07/MLMA_EOG/blob/main/Ueser_Independent_Logistic_Regression%26Perceptron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from google.colab import drive
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch
from torch.utils.data import Dataset, DataLoader
import pickle
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:

drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
main_dir = '/content/drive/MyDrive/S1Database/isolated/'
os.listdir(main_dir)

['ReadMe', '004', '002', '006', '003', '005', '001']

In [ ]:
# get data
def get_original_feature_label(path):
    # scale both vertical and horizontal between 0 and 1
    scaler = MinMaxScaler()
    feature = []
    label = []
    for i in os.listdir(path):
        a = pd.read_csv(str(path+i),names=["vertical","horizontal"])
        a = np.array(a)
        a = scaler.fit_transform(a)
        a = np.swapaxes(a,0,1)
        feature.append(a)
        if i.endswith("001.csv"):
            label.append(0)
        elif i.endswith("002.csv"):
            label.append(1)
        elif i.endswith("003.csv"):
            label.append(2)
        elif i.endswith("004.csv"):
            label.append(3)
        elif i.endswith("005.csv"):
            label.append(4)
        elif i.endswith("006.csv"):
            label.append(5)
        elif i.endswith("007.csv"):
            label.append(6)
        elif i.endswith("008.csv"):
            label.append(7)
        elif i.endswith("009.csv"):
            label.append(8)
        elif i.endswith("010.csv"):
            label.append(9)
        elif i.endswith("011.csv"):
            label.append(10)
        elif i.endswith("012.csv"):
            label.append(11)
    
    return feature,label

In [ ]:
# Define custom dataset class
class MYDataset(Dataset):
    def __init__(self, feature,label):
      self.feature = []
      self.label = []
      for i in range(len(feature)):
        self.feature.append(feature[i])
        self.label.append(label[i])

    def __len__(self):
        return len(self.label)

    def __getitem__(self, idx):
        f = self.feature[idx]
        f = f[np.newaxis,:,:]
        l = self.label[idx]
        return f, l


# Split on patient level - User-independent

In [ ]:
patient = ["001",'002','003','004','005']
train_feature = []
train_label = []
for i in patient:
    f,l = get_original_feature_label(str(main_dir+i+"/isolated_strokes/"))
    #print(len(f),f[0].shape)
    for x in f:
        train_feature.append(x)         
    for x in l:
        train_label.append(x)
print(len(train_feature))
train_data = MYDataset(train_feature,train_label)
train_dataloader  = DataLoader(train_data, batch_size=16, shuffle=True)


patient = ["006"]
test_feature = []
test_label = []
for i in patient:
    f,l = get_original_feature_label(str(main_dir+i+"/isolated_strokes/"))
    #print(len(f),f[0].shape)
    for x in f:
        test_feature.append(x)         
    for x in l:
        test_label.append(x)
print(len(test_feature))
test_data = MYDataset(test_feature,test_label)
test_dataloader  = DataLoader(test_data, batch_size=16, shuffle=True)

603
121


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, f1_score
from sklearn.preprocessing import label_binarize
import numpy as np

# Set input and output dimensions
input_dim = 2 * 1250
output_dim = 12

# Flatten the input features
train_feature_flat = [x.flatten() for x in train_feature]
test_feature_flat = [x.flatten() for x in test_feature]

# Create a logistic regression model
model = LogisticRegression(max_iter=1000)

# Train the model
model.fit(train_feature_flat, train_label)

# Make predictions on test data
test_predictions = model.predict(test_feature_flat)

# Get predicted probabilities
test_probabilities = model.predict_proba(test_feature_flat)

# Calculate accuracy
accuracy = accuracy_score(test_label, test_predictions)

# Calculate confusion matrix
confusion = confusion_matrix(test_label, test_predictions)

# Binarize the labels and predicted probabilities
test_label_binary = label_binarize(test_label, classes=np.arange(output_dim))
test_probabilities_binary = test_probabilities

# Calculate AUC for each class
auc = roc_auc_score(test_label_binary, test_probabilities_binary, multi_class='ovr')

# Calculate F1 score
f1 = f1_score(test_label, test_predictions, average='weighted')

print("Accuracy:", accuracy)
print("Confusion Matrix:", confusion)
print("AUC:", auc)
print("F1 Score:", f1)


Accuracy: 0.7851239669421488
Confusion Matrix: [[ 4  0  0  0  4  1  0  0  0  1  0  0]
 [ 0  4  0  6  0  0  0  0  0  0  0  0]
 [ 0  0 10  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  9  0  0  0  0  1  0  0  0]
 [ 0  0  0  0 11  0  0  0  0  0  0  0]
 [ 0  0  0  0  0 10  0  0  0  0  0  0]
 [ 0  0  0  0  0  0 10  0  0  0  0  0]
 [ 0  0  0  0  0  0  0 10  0  0  0  0]
 [ 5  0  0  3  0  0  0  0  2  0  0  0]
 [ 0  0  0  0  0  0  0  0  0 10  0  0]
 [ 0  0  0  0  0  0  3  0  0  0  7  0]
 [ 0  2  0  0  0  0  0  0  0  0  0  8]]
AUC: 0.9835207107934382
F1 Score: 0.7676955979921019


/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [11]:
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, roc_auc_score
import numpy as np
from sklearn.preprocessing import label_binarize
from sklearn.calibration import CalibratedClassifierCV

# Set input and output dimensions
input_dim = 2 * 1250
output_dim = 12

# Flatten the input features
train_feature_flat = [x.flatten() for x in train_feature]
test_feature_flat = [x.flatten() for x in test_feature]

# Create a perceptron model
model = Perceptron()

# Calibrate the classifier to get predicted probabilities
calibrated_model = CalibratedClassifierCV(model, method='sigmoid', cv=5)
calibrated_model.fit(train_feature_flat, train_label)

# Make predictions on test data
test_predictions = calibrated_model.predict(test_feature_flat)

# Calculate accuracy
accuracy = accuracy_score(test_label, test_predictions)

# Calculate confusion matrix
confusion = confusion_matrix(test_label, test_predictions)

# Calculate F1 score
f1 = f1_score(test_label, test_predictions, average='weighted')

# Calculate probabilities for AUC
test_probabilities = calibrated_model.predict_proba(test_feature_flat)

# Binarize the test labels for multiclass AUC calculation
test_label_binarized = label_binarize(test_label, classes=np.arange(output_dim))

# Calculate AUC
auc = roc_auc_score(test_label_binarized, test_probabilities, multi_class='ovo', average='weighted')

print("Accuracy:", accuracy)
print("Confusion Matrix:", confusion)
print("F1 Score:", f1)
print("AUC:", auc)


Accuracy: 0.6859504132231405
Confusion Matrix: [[ 1  0  0  0  6  1  1  0  0  1  0  0]
 [ 0  1  0  6  0  2  0  1  0  0  0  0]
 [ 0  0 10  0  0  0  0  0  0  0  0  0]
 [ 1  0  0  8  0  1  0  0  0  0  0  0]
 [ 0  0  0  0 11  0  0  0  0  0  0  0]
 [ 0  0  0  0  0 10  0  0  0  0  0  0]
 [ 0  0  0  0  0  0 10  0  0  0  0  0]
 [ 0  0  0  0  0  0  0 10  0  0  0  0]
 [ 7  0  0  2  0  0  0  0  1  0  0  0]
 [ 0  0  0  0  0  0  0  0  0 10  0  0]
 [ 0  0  0  0  0  0  8  0  0  0  2  0]
 [ 0  0  0  0  0  1  0  0  0  0  0  9]]
F1 Score: 0.6300569159308341
AUC: 0.9582296045105962
